In [ ]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display
from __init__ import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest


In [ ]:
symbol = 'EURUSD'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20 
ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()

display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20'])
display(fig)

In [ ]:
def get_signal(x):
    if x['open'] > x['sma_20']:
        return 'buy'
    elif x['open'] < x['sma_20']:
        return 'sell'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc

In [ ]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots 
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal 
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['signal'] == 'sell':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['signal'] == 'buy':
            orders.close_trade(trade)

In [ ]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

In [ ]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [ ]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20'])
backtest_fig

In [ ]:
evaluate_backtest(bt.trades)

In [ ]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [ ]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20'])
backtest_fig

In [ ]:
evaluate_backtest(bt.trades)
